<h1>Frequent itemset</h1>
begin met basics

In [1]:
import pandas as pd
import pyodbc

sales = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\\Users\\Burto\\Downloads\\GO-databases DEDS-week 4 & 5\\HC code\Week 10\\GO-databases schoon\\go_sales_schoon.accdb;')

order_details = pd.read_sql("SELECT * FROM order_details", sales)
order_header = pd.read_sql("SELECT * FROM order_header", sales)
product = pd.read_sql("SELECT * FROM product", sales)
product_type = pd.read_sql("SELECT * FROM product_type", sales)

C:\Users\Burto\AppData\Local\Temp\ipykernel_9844\3928950512.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_details = pd.read_sql("SELECT * FROM order_details", sales)
C:\Users\Burto\AppData\Local\Temp\ipykernel_9844\3928950512.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_header = pd.read_sql("SELECT * FROM order_header", sales)
C:\Users\Burto\AppData\Local\Temp\ipykernel_9844\3928950512.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("SELECT * FROM product", sales)
C:\Users\

In [2]:
#controleer hoeveelheid orders
len(order_header)


5360

Merge tabellen en selecteer alleen relevante kolommen

In [4]:
df = order_details.merge(product, on='PRODUCT_NUMBER')
df = df.merge(order_header, on='ORDER_NUMBER')

df = df[['ORDER_NUMBER', 'PRODUCT_NAME']]

Maak een basket

In [5]:
basket = pd.crosstab(index=df['ORDER_NUMBER'], columns=df['PRODUCT_NAME'])

Geburuik apriori om frequent itemsets te vinden.<br/>
Reden min_support = 0.005: er zijn 5360 bestellingen. Met een min_support van 0.005 moeten er minimaal 27 bestellingen zijn van de producten. Dit scheelt in theorie best wel veel tijd in het berekenen van de frequente itemsets.</br>
Reden lift min_threshold=1 is gekozen vanwege de betekenis van lift. Als de waarde groter is dan 1 betekent jet dat er een sterkere associatie  is tussen de items.

In [8]:
# gebruik apriori om frequent itemsets te vinden
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket, min_support=0.005, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# sorteer de resultaten op lift
rules.sort_values('lift', ascending=False, inplace=True)

# print de resultaten
print(rules.head())


C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                                            antecedents  \
8633                (Lady Hailstorm Titanium Woods Set)   
8632  (Lady Hailstorm Titanium Irons, Blue Steel Max...   
8634                    (Lady Hailstorm Titanium Irons)   
8631  (Blue Steel Max Putter, Lady Hailstorm Titaniu...   
9338    (Firefly Charger, Canyon Mule Journey Backpack)   

                                            consequents  antecedent support  \
8633  (Lady Hailstorm Titanium Irons, Blue Steel Max...            0.059142   
8632                (Lady Hailstorm Titanium Woods Set)            0.019590   
8634  (Blue Steel Max Putter, Lady Hailstorm Titaniu...            0.054664   
8631                    (Lady Hailstorm Titanium Irons)            0.022761   
9338                                    (Husky Rope 60)            0.012687   

      consequent support   support  confidence      lift  leverage  \
8633            0.019590  0.010448    0.176656  9.017876  0.009289   
8632            0.059142  0.01

De output van de code is de resulterende associatieregel tussen twee producten (of sets van producten), inclusief de frequentie van de ondersteunende regel, de confidence van de regel, de lift, de leverage, de conviction en de Zhang's metric.

De "antecedents" zijn de set van producten die aanwezig zijn in de "basket" van de klanten die het product kochten en de "consequents" zijn de set van producten die samen met de "antecedents" worden gekocht.

De "antecedent support" en "consequent support" geven de frequentie aan waarmee de sets van producten in de data voorkomen.

De "support" geeft aan hoe vaak de "antecedents" en "consequents" samen in een transactie voorkomen.

De "confidence" geeft de mate van zekerheid aan dat de "consequents" worden gekocht als de "antecedents" al in de "basket" zitten.

De "lift" geeft aan hoeveel meer kans er is dat de "consequents" samen met de "antecedents" worden gekocht dan verwacht als de producten onafhankelijk zouden zijn.

De "leverage" meet het verschil tussen de waargenomen frequentie van "antecedents" en "consequents" in de data en de frequentie die zou worden verwacht als de producten onafhankelijk zouden zijn.

De "conviction" geeft aan hoeveel sterker de "antecedents" de "consequents" voorspellen dan wanneer de "consequents" onafhankelijk zouden zijn.

De "Zhang's metric" is een maatstaf voor de afhankelijkheid tussen de "antecedents" en "consequents" en geeft een waarde tussen -1 en 1, waarbij 1 betekent dat de "antecedents" en "consequents" perfect afhankelijk zijn en -1 betekent dat ze perfect onafhankelijk zijn.